# Examples

## Conversion ase <> cfg files

In [1]:
from pathlib import Path
from src.utils import atoms2cfg, cfg2atoms # Call the code installed.
# from utils import atoms2cfg, cfg2atoms # development
from ase.io import read

In [2]:
current = Path().cwd()
outcar_path = current.parents[1] / "mlip-2/test/examples/00.convert_vasp_outcar/OUTCAR"
traj = read(outcar_path, index=":")
file = "./example/test.cfg"
for i,atoms in enumerate(traj):
    append = False if i==0 else True
    atoms2cfg(atoms, file=file, append=append)
    

FileNotFoundError: [Errno 2] No such file or directory: '/home/tomo/git/mtp4py/mlip-2/test/examples/00.convert_vasp_outcar/OUTCAR'

In [ ]:
a = list(cfg2atoms(file="./example/test.cfg"))
b = list(cfg2atoms(file="./example/executable.cfg", symbols={1: "Ag"}))

In [4]:
a

[Atoms(symbols='Ag32', pbc=True, cell=[[8.458526959, 1.656720733, 1.673168563], [1.573491987, 8.821430406, -1.494376014], [1.591543182, -1.495192313, 8.82509342]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.464367542, 1.654782691, 1.677485389], [1.570870501, 8.830852271, -1.499033421], [1.596387395, -1.500887298, 8.836299561]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.466931764, 1.653931823, 1.679380628], [1.569719577, 8.834988801, -1.501078187], [1.598514175, -1.503387596, 8.841219451]], calculator=SinglePointCalculator(...))]

In [5]:
b

[Atoms(symbols='Ag32', pbc=True, cell=[[8.458527, 1.656721, 1.673169], [1.573492, 8.82143, -1.494376], [1.591543, -1.495192, 8.825093]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.464368, 1.654783, 1.677485], [1.570871, 8.830852, -1.499033], [1.596387, -1.500887, 8.8363]], calculator=SinglePointCalculator(...)),
 Atoms(symbols='Ag32', pbc=True, cell=[[8.466932, 1.653932, 1.679381], [1.56972, 8.834989, -1.501078], [1.598514, -1.503388, 8.841219]], calculator=SinglePointCalculator(...))]

## Training

In [6]:
from src.training import MtpTraining # Call the code installed.
from pathlib import Path
# from training import MtpTraining # development

In [7]:
current = Path().cwd()
repo_path = current.parents[
    next(
        i
        for i in range(len(current.parents))
        if current.parents[i].name == "mtp4py"
    )
]
mtp_path = repo_path.parent / "mlip-2"

setting_path = mtp_path / "test/examples/01.train/06.mtp"
exe_path = mtp_path / "bin/mlp"
cfg_path = mtp_path / "test/examples/01.train/train.cfg"
dataset_name = cfg_path.name.split(".")[0]
output_path = repo_path / f"src/example/pot_{dataset_name}.almtp"

mtp = MtpTraining(setting_path, exe_path, cfg_path, output_path)
mtp.run()

/home/tomoyuki/git/mlip-2/bin/mlp train /home/tomoyuki/git/mlip-2/test/examples/01.train/06.mtp /home/tomoyuki/git/mlip-2/test/examples/01.train/train.cfg --trained-pot-name=/home/tomoyuki/git/mtp4py/src/example/pot_train.almtp --max-iter=100
MTPR from /home/tomoyuki/git/mlip-2/test/examples/01.train/06.mtp, Database: /home/tomoyuki/git/mlip-2/test/examples/01.train/train.cfg
Random initialization of radial coefficients
Rescaling...
   scaling = 0.833333333333333, condition number = 848.958997693705
   scaling = 0.909090909090909, condition number = 926.136632535035
   scaling = 1, condition number = 1018.74930264829
   scaling = 1.1, condition number = 1120.62333271922
   scaling = 1.2, condition number = 1222.49743610979
Rescaling to 0.833333333333333... done
Rescaling...
   scaling = 0.694444444444445, condition number = 707.468343705292
   scaling = 0.757575757575758, condition number = 771.782445907155
   scaling = 0.833333333333333, condition number = 848.959489340706
   scaling 

## Infer

In [8]:
from src.training import MtpTraining # Call the code installed.
# from ase_calculator_mtp import AseCalculatorMtp # development
from ase.optimize import FIRE


In [9]:
calc = AseCalculatorMtp(output_path)
cfgs = list(cfg2atoms(cfg_path))
atoms = cfgs[0].copy()
atoms.calc = calc
with FIRE(atoms, "./example/test_traj.traj") as fire:
    fire.run()


      Step     Time          Energy          fmax
FIRE:    0 16:51:01      -37.125699        0.479022
FIRE:    1 16:51:01      -37.140107        0.466766
FIRE:    2 16:51:01      -37.167052        0.441238
FIRE:    3 16:51:01      -37.202974        0.401957
FIRE:    4 16:51:01      -37.243016        0.347998
FIRE:    5 16:51:01      -37.281516        0.278577
FIRE:    6 16:51:01      -37.312666        0.194328
FIRE:    7 16:51:01      -37.330819        0.104855
FIRE:    8 16:51:01      -37.332275        0.108342
FIRE:    9 16:51:01      -37.332513        0.106365
FIRE:   10 16:51:01      -37.332974        0.102457
FIRE:   11 16:51:01      -37.333626        0.096707
FIRE:   12 16:51:01      -37.334424        0.089248
FIRE:   13 16:51:01      -37.335315        0.080257
FIRE:   14 16:51:01      -37.336240        0.069951
FIRE:   15 16:51:01      -37.337140        0.058597
FIRE:   16 16:51:01      -37.338040        0.046247


# ase db

In [9]:
from pathlib import Path
from ase.io import read
from ase.build import sort
from ase.calculators.emt import EMT
from ase.optimize.fire import FIRE
import numpy as np
from ase.db import connect
from ase import Atom

if Path('./test_ase_db.db').exists():
    Path('./test_ase_db.db').unlink()
db = connect('test_ase_db.db')

def save_to_db(atoms, opt_id):
    # 構造とエネルギーをデータベースに保存
    fmax = np.max(np.linalg.norm(atoms.get_forces(), axis=1))
    db.write(atoms, data={"opt_id":opt_id})

from ase.calculators.lammpslib import LAMMPSlib
lammps_command = [
    "pair_style meam",
    f"pair_coeff * * ./library.meam Pt Fe ./PtFe.meam Pt Fe",
    f"mass 1 {Atom('Pt').mass}",
    f"mass 2 {Atom('Fe').mass}",
]

calc = LAMMPSlib(lmpcmds=lammps_command, atoms_types={"Fe": 2, "Pt": 1}, keep_alive=True)

def opt(FePt, opt_id):
    # for lc in [1.1, 1, 0.9]:
    for lc in [1]:
        FePt = sort(FePt)
        sc = FePt.get_scaled_positions()
        FePt.cell *= lc
        FePt.set_scaled_positions(sc)
        FePt.positions += np.random.normal(0, 0.4, 3*len(FePt)).reshape(-1, 3)

        FePt.calc = calc
        opt = FIRE(FePt)
        opt.attach(save_to_db, interval=1, atoms=FePt, opt_id=opt_id)
        opt.run(fmax=0.05)

FePt = read("./FePt.cif")
FePt = sort(FePt)


FePt_rpt = FePt.repeat([2,2,2])
#FePt_rpt.symbols[FePt_rpt.symbols=="Fe"] = "Pt"
FePt_rpt = sort(FePt_rpt)
print(FePt_rpt.symbols)
opt(FePt_rpt, 1)

FePt_rpt = FePt.repeat([2,2,2])
ids = np.random.choice(np.where(FePt_rpt.symbols == "Pt")[0], 3)
FePt_rpt.symbols[ids] = "Fe"
opt(FePt_rpt, 3)

FePt_rpt = FePt.repeat([2,2,2])
ids = np.random.choice(np.where(FePt_rpt.symbols == "Fe")[0], 3)
FePt_rpt.symbols[ids] = "Pt"
print(FePt_rpt)
opt(FePt_rpt, 4)


Fe8Pt8
      Step     Time          Energy          fmax
FIRE:    0 09:23:17       82.839131      355.303287
FIRE:    1 09:23:17       17.205578      126.126216
FIRE:    2 09:23:17      -10.398573       56.138083
FIRE:    3 09:23:17      -25.215018       38.849958
FIRE:    4 09:23:17      -35.362660       43.489360
FIRE:    5 09:23:17      -43.324526       42.005841
FIRE:    6 09:23:17      -49.628424       35.899298
FIRE:    7 09:23:17      -54.370203       28.590053
FIRE:    8 09:23:17      -57.835139       21.367120
FIRE:    9 09:23:17      -60.598349       22.624501
FIRE:   10 09:23:17      -63.328721       22.662181
FIRE:   11 09:23:17      -66.285291       18.962709
FIRE:   12 09:23:17      -69.322751       13.654341
FIRE:   13 09:23:17      -71.689481        8.942998
FIRE:   14 09:23:17      -73.187511        6.634095
FIRE:   15 09:23:17      -74.014682        6.680485
FIRE:   16 09:23:17      -74.494275        7.378388
FIRE:   17 09:23:17      -75.007210        8.073472
FIRE:  

In [10]:
def get_atoms_data():
    for row in db.select("fmax<1"):
        yield row.toatoms()

In [20]:
atoms_list = list(get_atoms_data())
import importlib
import src.lj_optimizer as ljop
importlib.reload(ljop)


init_eps_dict = {"Pt":1, "Fe" : 1}
init_sigma_dict = {"Pt":3, "Fe" : 2.3}
#lj = ljop.LJTrainerLB(init_eps_dict, init_sigma_dict)
lj = ljop.LJTrainer(None, None)
eps, sig, chempot = lj.run_minimize(minimized_by="ef", labelled_list_atoms=atoms_list, maxiter=10, save_to_file="./washoi.yml", e_weight=0.1)
eps, sig, chempot

2024-09-06 09:47:13,238 - INFO - optimization is performed with ef, with log_scale False.
2024-09-06 09:47:13,253 - INFO - estimated rough chemical potentials
2024-09-06 09:47:13,253 - INFO - {np.str_('Fe'): np.float64(-2.0838431091116045), np.str_('Pt'): np.float64(-1.5793072486062878)}
2024-09-06 09:47:13,260 - INFO - objective: [e, f], epsilon:[('Fe', 'Fe'), ('Fe', 'Pt'), ('Pt', 'Pt')], sigma:[('Fe', 'Fe'), ('Fe', 'Pt'), ('Pt', 'Pt')], chempot:[('Fe', 'Fe'), ('Fe', 'Pt'), ('Pt', 'Pt')]
2024-09-06 09:47:13,307 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,350 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,395 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,437 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]


{'Fe': 1, 'Pt': 2}


2024-09-06 09:47:13,481 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,524 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,568 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,613 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,661 - INFO - 550.21, [1589.35, 434.75], [1.0419, 0.9158, 0.7897], [2.3, 2.3, 2.3], [-0.521, -0.395]
2024-09-06 09:47:13,707 - INFO - 5204.19, [3784.13, 5361.97], [0.6053, 0.532, 0.4588], [2.8, 1.96, 1.96], [-0.303, -0.229]
2024-09-06 09:47:13,749 - INFO - 5204.19, [3784.13, 5361.97], [0.6053, 0.532, 0.4588], [2.8, 1.96, 1.96], [-0.303, -0.229]
2024-09-06 09:47:13,791 - INFO - 5204.19, [3784.13, 5361.97], [0.6053, 0.532, 0.4588], [2.8, 1.96, 1.96], [-0.303, -0.229]
2024-09-06 09:47:13,838 - INFO 

({('Fe', 'Fe'): 0.7874058714702953,
  ('Fe', 'Pt'): 0.7582110322334289,
  ('Pt', 'Pt'): 0.7091884501728541},
 {('Fe', 'Fe'): 2.319710235680519,
  ('Fe', 'Pt'): 2.2505044381340005,
  ('Pt', 'Pt'): 2.208999554192122},
 {'Fe': -0.44640055911223453, 'Pt': -0.341326817131831})

In [17]:
atoms_list = list(get_atoms_data())
from src.ase_calculator_mtp import LammpsLJBuilder
from plotly import graph_objects as go
x= [atoms.get_potential_energy() for atoms in atoms_list]
init_eps_dict = {"Pt":1.3, "Fe" : 1.8}
init_sigma_dict = {"Pt":2.5, "Fe" : 2.0}
# init_eps_dict=eps
# init_sigma_dict=sig

def get_lj_engs():
    for atoms in atoms_list:
        _atoms = atoms.copy()
        _atoms.calc = LammpsLJBuilder().get_calculator_from_file(path_to_yaml="./washoi.yml")
        eng = _atoms.get_potential_energy()
        yield eng
y= [i for i in get_lj_engs()]

x = np.array(x)
y = np.array(y)
print(len(x))
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=x, y=y)
)


67


In [18]:
atoms_list = list(get_atoms_data())
from plotly import graph_objects as go
x= [atoms.get_forces() for atoms in atoms_list]
init_eps_dict = {"Pt":2, "Fe" : 1.5}
init_sigma_dict = {"Pt":2.8, "Fe" : 2.1}
# init_eps_dict=eps
# init_sigma_dict=sig

def get_lj_engs():
    for atoms in atoms_list:
        _atoms = atoms.copy()
        _atoms.calc = LammpsLJBuilder().get_calculator_from_file(path_to_yaml="./washoi.yml")
        eng = _atoms.get_forces()
        yield eng
y= [i for i in get_lj_engs()]

x = np.array(x).reshape(1, -1)[0]
y = np.array(y).reshape(1, -1)[0]

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=x[abs(y) < 1], y=y[abs(y) < 1])
)


In [85]:
np.array([atoms.get_forces() for atoms in atoms_list]).reshape(1, -1)[0]

array([-0.1578861 ,  0.24526497, -0.10827696, ..., -0.01679502,
       -0.01017749, -0.0094713 ])

In [40]:
from sqlalchemy import create_engine, Column, Integer, Float, String, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import select

# Declarative Base の作成
Base = declarative_base()

# テーブル定義をクラスを使って行う
class System(Base):
    __tablename__ = 'systems'
    
    id = Column(Integer, primary_key=True)
    calculator = Column(String)
    fmax = Column(Float)
    # 他のカラムも定義できますが、すべてのカラムが含まれる前提で進めます

# データベースエンジンの作成
engine = create_engine('sqlite:///test_ase_db.db')
Session = sessionmaker(bind=engine)
session = Session()

# fmaxの上位2位、下位2位のデータを取得
subquery_asc = session.query(System.id, System.calculator, System.fmax, 
                             func.row_number().over(
                                 partition_by=System.calculator, 
                                 order_by=System.fmax.asc()).label('rank_asc')).subquery()

subquery_desc = session.query(System.id, System.calculator, System.fmax, 
                              func.row_number().over(
                                  partition_by=System.calculator, 
                                  order_by=System.fmax.desc()).label('rank_desc')).subquery()

# 全カラムを取得しつつ、上位2位、下位2位を取得
query = session.query(System.__table__).filter(
    subquery_asc.c.rank_asc <= 2
).union(
    session.query(System.__table__).filter(
        subquery_desc.c.rank_desc <= 2
    )
).order_by(System.calculator, System.fmax)

# クエリの実行
results = query.all()

# 結果の表示
for row in results:
    print(f"ID: {row.id}, Calculator: {row.calculator}, Fmax: {row.fmax}")

ID: 47, Calculator: lammpslib, Fmax: 0.03743799106791549
ID: 46, Calculator: lammpslib, Fmax: 0.08722484600406126
ID: 45, Calculator: lammpslib, Fmax: 0.1255164125942915
ID: 41, Calculator: lammpslib, Fmax: 0.14712573966730683
ID: 40, Calculator: lammpslib, Fmax: 0.16134134735936206
ID: 44, Calculator: lammpslib, Fmax: 0.16548470943024424
ID: 42, Calculator: lammpslib, Fmax: 0.16719952479857078
ID: 39, Calculator: lammpslib, Fmax: 0.17345725414022395
ID: 43, Calculator: lammpslib, Fmax: 0.17791651416272855
ID: 38, Calculator: lammpslib, Fmax: 0.1837720073347541
ID: 37, Calculator: lammpslib, Fmax: 0.20147874593990484
ID: 36, Calculator: lammpslib, Fmax: 0.24475230652072122
ID: 27, Calculator: lammpslib, Fmax: 0.24761973454863298
ID: 28, Calculator: lammpslib, Fmax: 0.27825091678214764
ID: 35, Calculator: lammpslib, Fmax: 0.28096687251337454
ID: 34, Calculator: lammpslib, Fmax: 0.3135421997593967
ID: 33, Calculator: lammpslib, Fmax: 0.3417702989117921
ID: 32, Calculator: lammpslib, Fmax

/tmp/ipykernel_7538/3630815270.py:7: MovedIn20Warning:

The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

/tmp/ipykernel_7538/3630815270.py:44: SAWarning:

SELECT statement has a cartesian product between FROM element(s) "systems" and FROM element "anon_2".  Apply join condition(s) between each element to resolve.

/tmp/ipykernel_7538/3630815270.py:44: SAWarning:

SELECT statement has a cartesian product between FROM element(s) "systems" and FROM element "anon_3".  Apply join condition(s) between each element to resolve.



In [41]:

cursor.execute("PRAGMA table_info(systems);")
columns = cursor.fetchall()
for col in columns:
    print(col[1])

NameError: name 'cursor' is not defined

In [49]:
import struct
from ase import Atoms
# バイナリデータを整数リストに変換
binary_data = b'\x08\x00\x00\x00\x00\x00\x00\x00'
num_values = len(binary_data) // 4  # assuming each value is 4 bytes (e.g., int32)
values = struct.unpack(f'{num_values}I', binary_data)  # 'I' for unsigned int
print(values)

(8, 0)
